In [0]:
import copy

H_TABLE = [
  [0xB1, 0x94, 0xBA, 0xC8, 0x0A, 0x08, 0xF5, 0x3B, 0x36, 0x6D, 0x00, 0x8E, 0x58, 0x4A, 0x5D, 0xE4],
  [0x85, 0x04, 0xFA, 0x9D, 0x1B, 0xB6, 0xC7, 0xAC, 0x25, 0x2E, 0x72, 0xC2, 0x02, 0xFD, 0xCE, 0x0D],
  [0x5B, 0xE3, 0xD6, 0x12, 0x17, 0xB9, 0x61, 0x81, 0xFE, 0x67, 0x86, 0xAD, 0x71, 0x6B, 0x89, 0x0B],
  [0x5C, 0xB0, 0xC0, 0xFF, 0x33, 0xC3, 0x56, 0xB8, 0x35, 0xC4, 0x05, 0xAE, 0xD8, 0xE0, 0x7F, 0x99],
  [0xE1, 0x2B, 0xDC, 0x1A, 0xE2, 0x82, 0x57, 0xEC, 0x70, 0x3F, 0xCC, 0xF0, 0x95, 0xEE, 0x8D, 0xF1],
  [0xC1, 0xAB, 0x76, 0x38, 0x9F, 0xE6, 0x78, 0xCA, 0xF7, 0xC6, 0xF8, 0x60, 0xD5, 0xBB, 0x9C, 0x4F],
  [0xF3, 0x3C, 0x65, 0x7B, 0x63, 0x7C, 0x30, 0x6A, 0xDD, 0x4E, 0xA7, 0x79, 0x9E, 0xB2, 0x3D, 0x31],
  [0x3E, 0x98, 0xB5, 0x6E, 0x27, 0xD3, 0xBC, 0xCF, 0x59, 0x1E, 0x18, 0x1F, 0x4C, 0x5A, 0xB7, 0x93],
  [0xE9, 0xDE, 0xE7, 0x2C, 0x8F, 0x0C, 0x0F, 0xA6, 0x2D, 0xDB, 0x49, 0xF4, 0x6F, 0x73, 0x96, 0x47],
  [0x06, 0x07, 0x53, 0x16, 0xED, 0x24, 0x7A, 0x37, 0x39, 0xCB, 0xA3, 0x83, 0x03, 0xA9, 0x8B, 0xF6],
  [0x92, 0xBD, 0x9B, 0x1C, 0xE5, 0xD1, 0x41, 0x01, 0x54, 0x45, 0xFB, 0xC9, 0x5E, 0x4D, 0x0E, 0xF2],
  [0x68, 0x20, 0x80, 0xAA, 0x22, 0x7D, 0x64, 0x2F, 0x26, 0x87, 0xF9, 0x34, 0x90, 0x40, 0x55, 0x11],
  [0xBE, 0x32, 0x97, 0x13, 0x43, 0xFC, 0x9A, 0x48, 0xA0, 0x2A, 0x88, 0x5F, 0x19, 0x4B, 0x09, 0xA1],
  [0x7E, 0xCD, 0xA4, 0xD0, 0x15, 0x44, 0xAF, 0x8C, 0xA5, 0x84, 0x50, 0xBF, 0x66, 0xD2, 0xE8, 0x8A],
  [0xA2, 0xD7, 0x46, 0x52, 0x42, 0xA8, 0xDF, 0xB3, 0x69, 0x74, 0xC5, 0x51, 0xEB, 0x23, 0x29, 0x21],
  [0xD4, 0xEF, 0xD9, 0xB4, 0x3A, 0x62, 0x28, 0x75, 0x91, 0x14, 0x10, 0xEA, 0x77, 0x6C, 0xDA, 0x1D]
]

def string_to_bit_array(raw_text):
  array = list()
  for char in raw_text:
    bin_val = bin_value(char, 8)
    array.extend([int(x) for x in list(bin_val)])
  return array

def bit_array_to_string(array):
  res = ''.join([chr(int(y, 2)) for y in [''.join([str(x) for x in _bytes]) for _bytes in nsplit(array, 8)]])
  return res

def bin_value(val, bitsize):
  bin_val = bin(val)[2:] if isinstance(val, int) else bin(ord(val))[2:]
  if len(bin_val) > bitsize:
    raise Exception('binary value larger than the expected size')
  while len(bin_val) < bitsize:
    bin_val = '0' + bin_val
  return bin_val

def nsplit(s, n):
  return [s[k:k + n] for k in range(0, len(s), n)]

def int_to_bit(n):
  return [int(digit) for digit in bin(n)[2:]]

def bit_to_int(value):
  return int(''.join(map(str, value)), 2)

In [0]:
class BeLT:
  def __init__(self, encode_key):
    encode_key = string_to_bit_array(encode_key)
    self.sub_keys = nsplit(encode_key, 32)

  @staticmethod
  def H_get_data_from_table(val):
    if '0x' in val:
      val = val[2:]
      if len(val) > 2:
        raise ValueError('cannot transfer data from table')
      elif len(val) == 1:
        return H_TABLE[int('0x0', 16)][int(val[0], 16)]
    return H_TABLE[int(val[0], 16)][int(val[1], 16)]

  @staticmethod
  def mod_32(a, b, is_plus=True):
    int_a = bit_to_int(a)
    int_b = bit_to_int(b)
    if is_plus:
      int_result = (int_a + int_b) % (2 ** 32)
    else:
      int_result = (int_a - int_b) % (2 ** 32)
    result = int_to_bit(int_result)
    while len(result) < 32:
      result.insert(0, 0)
    return result

  def G_transition(self, data, num):
    int_data = nsplit(data, 8)
    new_data = []
    for val in int_data:
      int_val = bit_to_int(val)
      hex_val = str(hex(int_val))
      new_data.append(self.H_get_data_from_table(hex_val))

    for i in range(len(new_data)):
      temp = int_to_bit(new_data[i])
      while len(temp) < 8:
        temp.insert(0, 0)
      new_data[i] = temp

    new_data = [item for sublist in new_data for item in sublist]

    int_s = bit_to_int(new_data)
    int_s = ((int_s >> (32 - num)) | (int_s << num)) & 0xFFFFFFFF
    result = int_to_bit(int_s)
    while len(result) < 32:
      result.insert(0, 0)
    return result

  @staticmethod
  def bit_xor(arr1, arr2):
    bit_s = []
    for index, item in enumerate(arr1):
      bit_s.append(arr1[index] ^ arr2[index])
    return bit_s

  def calc(self, i, minus, part, g_index, all_keys):
    sub_key = copy.deepcopy(all_keys[7 * i - minus - 1])
    temp = self.mod_32(part, sub_key)
    temp = self.G_transition(temp, g_index)
    return temp

  def encode(self, raw_text):
    raw_text_bits = string_to_bit_array(raw_text)
    raw_splitted_text = nsplit(raw_text_bits, 32)
    a = raw_splitted_text[0]
    b = raw_splitted_text[1]
    c = raw_splitted_text[2]
    d = raw_splitted_text[3]

    all_keys = self.sub_keys * 7

    for i in range(1, 9):
      temp = self.calc(i, minus=6, part=a, g_index=5, all_keys=all_keys)
      b = self.bit_xor(b, temp)

      temp = self.calc(i, minus=5, part=d, g_index=21, all_keys=all_keys)
      c = self.bit_xor(c, temp)

      temp = self.calc(i, minus=4, part=b, g_index=13, all_keys=all_keys)
      a = self.mod_32(a, temp, is_plus=False)

      sub_key = copy.deepcopy(all_keys[7 * i - 3 - 1])
      temp = self.mod_32(b, c)
      temp = self.mod_32(temp, sub_key)
      temp = self.G_transition(temp, 21)
      val_i = i % (2 ** 32)
      val_i = int_to_bit(val_i)
      while len(val_i) < 32:
        val_i.insert(0, 0)
      e = self.bit_xor(temp, val_i)

      b = self.mod_32(b, e)

      c = self.mod_32(c, e, is_plus=False)

      temp = self.calc(i, minus=2, part=c, g_index=13, all_keys=all_keys)
      d = self.mod_32(d, temp)

      temp = self.calc(i, minus=1, part=a, g_index=21, all_keys=all_keys)
      b = self.bit_xor(b, temp)

      temp = self.calc(i, minus=0, part=d, g_index=5, all_keys=all_keys)
      c = self.bit_xor(c, temp)

      a, b = b, a

      c, d = d, c

      b, c = c, b

    encoded_text = b + d + a + c
    encoded_text = bit_array_to_string(encoded_text)
    return encoded_text

  def decode(self, encoded_text):
    raw_text_bits = string_to_bit_array(encoded_text)
    raw_splitted_text = nsplit(raw_text_bits, 32)
    a = raw_splitted_text[0]
    b = raw_splitted_text[1]
    c = raw_splitted_text[2]
    d = raw_splitted_text[3]

    all_keys = self.sub_keys * 7

    for i in range(8, 0, -1):
      temp = self.calc(i, minus=0, part=a, g_index=5, all_keys=all_keys)
      b = self.bit_xor(b, temp)

      temp = self.calc(i, minus=1, part=d, g_index=21, all_keys=all_keys)
      c = self.bit_xor(c, temp)

      temp = self.calc(i, minus=2, part=b, g_index=13, all_keys=all_keys)
      a = self.mod_32(a, temp, is_plus=False)

      sub_key = copy.deepcopy(all_keys[7 * i - 3 - 1])
      temp = self.mod_32(b, c)
      temp = self.mod_32(temp, sub_key)
      temp = self.G_transition(temp, 21)
      val_i = i % (2 ** 32)
      val_i = int_to_bit(val_i)
      while len(val_i) < 32:
        val_i.insert(0, 0)
      e = self.bit_xor(temp, val_i)

      b = self.mod_32(b, e)

      c = self.mod_32(c, e, is_plus=False)

      temp = self.calc(i, minus=4, part=c, g_index=13, all_keys=all_keys)
      d = self.mod_32(d, temp)

      temp = self.calc(i, minus=5, part=a, g_index=21, all_keys=all_keys)
      b = self.bit_xor(b, temp)

      temp = self.calc(i, minus=6, part=d, g_index=5, all_keys=all_keys)
      c = self.bit_xor(c, temp)

      a, b = b, a

      c, d = d, c

      a, d = d, a

    decoded_text = c + a + d + b
    decoded_text = bit_array_to_string(decoded_text)
    return decoded_text

  def simple_change(self, raw_text, is_encode=True):
    text = []
    raw_text_splitted = nsplit(raw_text, 16)
    for part in raw_text_splitted:
      if is_encode:
        text.append(self.encode(part))
      else:
        text.append(self.decode(part))

    text = ''.join(text)
    return text

  def block_chaining(self, raw_text, sync, is_encode=True):
    text = []
    raw_text_splitted = nsplit(raw_text, 16)
    temp = self.encode(sync)
    for part in raw_text_splitted:
      if is_encode:
        res = bit_array_to_string(self.bit_xor(string_to_bit_array(temp), string_to_bit_array(part)))
        encoded_part = self.encode(res)
        text.append(encoded_part)
        temp = encoded_part
      else:
        bit_temp = string_to_bit_array(temp)
        ans = bit_array_to_string(self.bit_xor(string_to_bit_array(self.decode(part)), bit_temp))
        text.append(ans)
        temp = part

    text = ''.join(text)
    return text

  def cipher_feedback(self, raw_text, sync, is_encode=True):
    splitted_raw_text = nsplit(raw_text, 16)
    text = []
    temp = sync
    for part in splitted_raw_text:
      enc_temp = self.encode(temp)
      enc_sliced = enc_temp[:len(part)]
      ans = bit_array_to_string(self.bit_xor(string_to_bit_array(part), string_to_bit_array(enc_sliced)))
      text.append(ans)
      if is_encode:
        temp = ans
      else:
        temp = part

    text = ''.join(text)
    return text

  def counter(self, raw_text, sync):
    splitted_raw_text = nsplit(raw_text, 16)
    text = []
    s = self.encode(sync)
    ones = [1 for _ in range(128)]
    for part in splitted_raw_text:
      s = self.mod_32(string_to_bit_array(s), ones)
      s = self.encode(s)
      sliced_s = s[:len(part)]
      ans = bit_array_to_string(self.bit_xor(string_to_bit_array(part), string_to_bit_array(sliced_s)))
      text.append(ans)

    text = ''.join(text)
    return text

In [0]:
key = 'sanfksalkfn1esdkmfsdfd23j32fmsdl'
sync_send = '1a2b3c4d5d6e7f8g'
stb = BeLT(key)

text = 'asdfklsajdflkjfj'

In [7]:
enc_data = stb.simple_change(text, is_encode=True)
enc_data

'Þ@MÐ\x1c[Ä\x9d³\x08J\x11z*V\x17'

In [8]:
dec_data = stb.simple_change(enc_data, is_encode=False)
dec_data

'asdfklsajdflkjfj'

In [9]:
enc_data = stb.block_chaining(text, sync_send, is_encode=True)
enc_data

'$\x0bòRc°w&\x1bîûÛÅ\x9fSP'

In [10]:
dec_data = stb.block_chaining(enc_data, sync_send, is_encode=False)
dec_data

'asdfklsajdflkjfj'

In [11]:
enc_data = stb.cipher_feedback(text, sync_send, is_encode=True)
enc_data

'\x91¹l\x9fgá®Ú^\x9f[n\r\x0eTÇ'

In [12]:
dec_data = stb.cipher_feedback(enc_data, sync_send, is_encode=False)
dec_data

'asdfklsajdflkjfj'

In [13]:
enc_data = stb.counter(text, sync_send)
enc_data

'y\x1a6ÓaJ\x94Wx®¬~Ü7V\x0b'

In [14]:
dec_data = stb.counter(enc_data, sync_send)
dec_data

'asdfklsajdflkjfj'